In [1]:
# import FunctionalCollections
push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")
import Iterators
import Pipe
import Compat
import JLD
import DataStructures
import DataStructuresExtended
@everywhere using FunctionalCollections
@everywhere using Iterators
@everywhere using Pipe
@everywhere using Compat
@everywhere using JLD
@everywhere using DataStructures
@everywhere using DataStructuresExtended

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end



In [21]:
@everywhere const START_MARKER1 = Symbol("**START1**")
@everywhere const START_MARKER2 = Symbol("**START2**")
@everywhere const END_MARKER1 = Symbol("**END1**")
@everywhere const END_MARKER2 = Symbol("**END2**")
@everywhere typealias S Symbol
@everywhere typealias State{T} Tuple{T,T}


In [3]:
function load_counts(filename)
    counts = Dict{Tuple{Symbol,Symbol,Symbol},Int}()
    open(filename, "r") do fh
        for line in eachline(fh)
            word1,word2,word3,occurrences = split(line)
            trigram = (Symbol(word1),Symbol(word2),Symbol(word3))
            counts[trigram] = parse(Int,occurrences)
        end
    end
    counts
end

load_counts (generic function with 1 method)

In [4]:
"""
    Kneser-Ney estimate of a probability distribution. This is a version of
    back-off that counts how likely an n-gram is provided the n-1-gram had
    been seen in training. Extends the ProbDistI interface, requires a trigram
    FreqDist instance to train on. Optionally, a different from default discount
    value can be specified. The default discount is set to 0.75.
    #Adapted from: http://www.nltk.org/_modules/nltk/probability.html
"""
type KneserNeyProbDist{T}
    
    trigrams :: Accumulator{Tuple{T,T,T},Int}
    bigrams  :: Accumulator{Tuple{T,T},Int}
    trigrams_contain :: Accumulator{T,Int}
    discount :: Float64
    wordtypes_after :: Accumulator{Tuple{T,T},Int}
    wordtypes_before :: Accumulator{Tuple{T,T},Int}
end

In [5]:


"""
:param trigrams: The trigram frequency distribution upon which to base
    the estimation
:param discount: The discount applied when retrieving counts of
    trigrams
"""
function KneserNeyProbDist{T}(trigrams :: Dict{Tuple{T,T,T},Int}, discount=0.75)

    # helper dictionaries used to calculate probabilities
    trigrams_contain = counter(T)
    bigrams  = counter(Tuple{T,T})
    wordtypes_after = counter(Tuple{T,T})
    wordtypes_before = counter(Tuple{T,T})

    for ((w0, w1, w2),n) in trigrams
        push!(trigrams_contain, w1)
        push!(bigrams, (w0,w1), n)
        push!(wordtypes_after,(w0,w1))  
        push!(wordtypes_before,(w1,w2))
    end
    KneserNeyProbDist{T}(counter(trigrams), bigrams, trigrams_contain, discount, wordtypes_after, wordtypes_before)
end
    

function prob{T}(self::KneserNeyProbDist{T}, w0::T,w1::T, w2::T)
    # if the sample trigram was seen during training
    if (w0, w1,w2) in keys(self.trigrams)
        @assert self.trigrams[(w0,w1,w2)]>self.discount
        (self.trigrams[(w0,w1,w2)] - self.discount)/self.bigrams[(w0, w1)]
    # else if the 'rougher' environment was seen during training
    elseif (w0,w1) in keys(self.bigrams) && (w1,w2) in keys(self.wordtypes_before)
        aftr = self.wordtypes_after[(w0, w1)]
        bfr = self.wordtypes_before[(w1, w2)]

        # the probability left over from alphas
        leftover_prob = aftr * self.discount  / self.bigrams[(w0, w1)]
        @assert leftover_prob>0
        # the beta (including normalization)
        beta = bfr / (self.trigrams_contain[w1] - aftr)
        @assert beta>0
        leftover_prob * beta
    # else the sample was completely unseen during training
    else
        0.0
    end
end

prob (generic function with 1 method)

In [6]:
ccs = load_counts("results/data/books/train_books_corpus.3gram")
kn_lm = KneserNeyProbDist(ccs)
ccs=0
kn_lm.trigrams |> length

0

In [14]:
@time prob(kn_lm, symbol("'m"), :so, :fast)

8.129023866814073e-6

0.000044 seconds (10 allocations: 320 bytes)


8.129023866814073e-6

0.000022 seconds (10 allocations: 320 bytes)


In [12]:
using JuMP
using MathProgBase
using GLPKMathProgInterface
using Gurobi

  likely near /home/ubuntu/.julia/v0.5/GLPK/src/GLPK.jl:812


In [29]:
"""
returns the a vector of sets of node indexes, each set is a subtour
The First subtour returned is the nonconnected one -- the path
"""
function get_subtours(x::Matrix{JuMP.Variable})
    x_val = getValue(x)
    x_iis,x_jjs, _  = findnz(x_val .>= 1 - 1e-6) #It just has to be close to 1 to be true
    nodes_chain = Dict([ii=>jj for (ii,jj) in zip(x_iis,x_jjs)])

    subtours = IntSet[]
    ii = START_NODE_INDEX
    push!(subtours, IntSet(END_NODE_INDEX)) #The END Node is always in the same subtour as the start node
    while(true)
        while(true) #Cycle through current subtour
            
            push!(subtours[end],ii) 
            
            jj = nodes_chain[ii]
            println(ii," ", jj)
            delete!(nodes_chain,ii)
            if jj∈subtours[end] 
                break 
            end
            ii=jj
        end   

        if length(nodes_chain)>0
            ii = first(keys(nodes_chain)) #start new subtour
            push!(subtours,IntSet())
        else
            break
        end
    end
    subtours
end

"""
Core tour goes from start to end,
Does not include the end node in the tour
"""
function get_coretours(x_links)
    nodes_chain = Dict([ii=>jj for (ii,jj) in x_links])

    coretour = IntSet()
    ii = START_NODE_INDEX
    while(ii!=END_NODE_INDEX)            
        push!(coretour,ii) 
        jj = nodes_chain[ii]
    end   
    coretour
end

function get_hyperclass(subtour)
    tour_hyper_class_nodes=IntSet()
    for ii in subtour
        w1,w2 = nodes[ii]
        #println(unordered_markers[w1])
        class_nodes = node_indexes_for_1st[w1]
        union!(tour_hyper_class_nodes,class_nodes) 
    end
    tour_hyper_class_nodes
end


get_hyperclass (generic function with 1 method)

In [25]:
#test_bag = shuffle(corpus[1022]) #length 28
#test_bag = shuffle(corpus[1028]) #length 20  (Gurodi 4.2, GLTK : 2064.6 seconds)
#test_bag = shuffle(corpus[1122]) #length 19 
#est_bag = shuffle(corpus[1000]) #length 17 (Gurbodi: 5.8 sconds. GLTK:  569.9seconds)
#test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy" ,"of","manners","first","performed","in","1892", "."])
#test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy" ,"of","manners","."])
test_bag =  shuffle(["this","is","the" ,"basis","of","a","fine","comedy", "."])
#^length 9, Gurodi 1.0, GLTK 5.3
#test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy", "."])
#test_bag =  shuffle(["it", "is", "so", "very", "good", "."])
#test_bag =  shuffle(["it", "is", "very", "good", "."])
#test_bag =  shuffle(["it", "is", "good", "."])
#test_bag =  ["no", "way", "."]
#test_bag =  shuffle(["no", "."])
#@time best_order(test_bag, lm, mem_limit=1000)
test_bag = map(symbol, test_bag)

9-element Array{Symbol,1}:
 :this  
 :a     
 :fine  
 :basis 
 :comedy
 :.     
 :the   
 :of    
 :is    

In [30]:
function lm(w0,w1,w2)
    prob(kn_lm,w0,w1,w2)
end


tic()
m=Model(solver=GurobiSolver())
#m=Model(solver=GLPKSolverMIP())



unordered_words  = test_bag
unordered_markers = [START_MARKER1; START_MARKER2; END_MARKER1; END_MARKER2; unordered_words...]
#Note that this lacks END_MARKER2
const START_NODE_INDEX = 1
const END_NODE_INDEX = 2

nodes = State{Int}[] #Named by word index

node_indexes_for_1st = Dict{Int, Vector{Int}}()
node_indexes_for_2nd = Dict{Int, Vector{Int}}()

function add_node!(ii,jj)
    push!(nodes, (ii,jj))

    node_indexes_for_i_1st = get!(()->Int[], node_indexes_for_1st, ii)
    push!(node_indexes_for_i_1st, length(nodes))

    node_indexes_for_j_2nd = get!(()->Int[], node_indexes_for_2nd, jj) 
    push!(node_indexes_for_j_2nd, length(nodes))
    #println("node:$(length(nodes)) |  $(unordered_markers[ii])($ii), $(unordered_markers[jj])($jj)")
end

add_node!(1, 2) #That is START_MARKER1-> START_MARKER2
add_node!(3, 4) #That is END_MARKER1-> END_MARKER2

for ii in 1:length(unordered_markers)
    wi = unordered_markers[ii]
    if wi∈(END_MARKER1,START_MARKER1) continue end  #Covered these
        
    for jj in [1:length(unordered_markers)]
        if ii==jj continue end
        wj = unordered_markers[jj]
        if wj∉(START_MARKER1,START_MARKER2,END_MARKER2)
            #but wj can be  END_MARKER1
            add_node!(ii,jj)
        end
    end
end

@defVar(m, x[1:length(nodes), 1:length(nodes)], Bin)

#If you enter a node you must also leave it
for (cc, center_node) in enumerate(nodes)
    if cc ∉ (START_NODE_INDEX, END_NODE_INDEX) #the beginning and end done have this requiement
        #Everything that enters this node, must leave this node
        @addConstraint(m, sum{x[ii,cc], ii=1:length(nodes)} == sum{x[cc,jj], jj=1:length(nodes)})
    end
end

for class_index in 1:length(unordered_markers)
    w_class =  unordered_markers[class_index]
    if w_class∉(START_MARKER1,START_MARKER2)
        #Not rquired to make a transition so that START_MARKER1 or 2 ever occur in second position
        jjs = node_indexes_for_2nd[class_index]
        @addConstraint(m, sum{x[ii,jj],ii=1:length(nodes), jj=jjs}==1)
    end
    
    #The following constraint is not required, as if it shows uo in the second position other rules make it certain to show up in the first
    #if w_class∉(END_MARKER1,END_MARKER2)
    #    #Not rquired to make a transition so that END_MARKER1 or 2 ever occur in second position
    #    iis=node_indexes_for_1st[class_index]
    #    @addConstraint(m, sum{x[ii,jj], ii=iis,jj=1:length(nodes)}==1)
    #    rules[w_class*"*"]=Set(product(iis,1:length(nodes)))
    #end
end

banned_trans_prob = Set{State{Int}}()
banned_trans_state = Set{State{Int}}()
log_trans_prob = spzeros(length(nodes), length(nodes))
for (from_node_index, from_node) in enumerate(nodes)
    w1 = unordered_markers[from_node[1]]
    w2 = unordered_markers[from_node[2]]
    #If what was in the second state element does not end up in the first state element then it is not allowed.
    can_transition_to = Set(get(node_indexes_for_1st, from_node[2], Int[]))
        #You can transition to any node which has your second element as its first element
    for (to_node_index, to_node) in enumerate(nodes)
        if to_node_index in can_transition_to
            @assert(from_node[2]==to_node[1])
            w3= unordered_markers[to_node[2]]
            log_tp = log(lm(w1,w2,w3))
            if log_tp>-Inf
                log_trans_prob[from_node_index, to_node_index] = log_tp
                continue
            else
                #Banned as prob zero transitions not allowed
                @addConstraint(m, x[from_node_index,to_node_index]==0)
                push!(banned_trans_prob, (from_node_index,to_node_index))
            end
            
        else
            #It is not a legal transition
            @addConstraint(m, x[from_node_index,to_node_index]==0)
            push!(banned_trans_state, (from_node_index,to_node_index))
        end
    end
end



function eleminate_subtours(cb)
    x_val = getValue(x)
    x_iis,x_jjs, _  = findnz(x_val .>= 1 - 1e-6) #It just has to be close to 1 to be true
    x_links=zip(x_iis,x_jjs)
    coretour = get_coretours(x_links)
    if length(coretour)==length(x_links)
        #no subtour to eleminate
        return
    end
    core_hyperclass = get_hyperclass(coretour)
    
    #Also the First subtour must go to one of the other subtours 
    arcs_outof_coretour = AffExpr()
    for ii in core_hyperclass
        for jj in 1:length(nodes)
            if jj∉core_hyperclass
                arcs_outof_coretour += x[ii,jj]
            end
        end
    end
    @addLazyConstraint(cb, arcs_outof_coretour >=1)
end

addLazyCallback(m, eleminate_subtours,fractional=false)

#new Linear way using logprobs
@setObjective(m, Max, sum{log_trans_prob[i,j]*x[i,j], i=1:length(nodes), j=1:length(nodes)})

toc()
@printval length(unordered_markers)
@printval length(m.linconstr)
log_trans_prob


elapsed time: 0.

103x103 sparse matrix with 516 Float64 entries:
	[68 ,   2]  =  -1.63369e-8
	[1  ,   4]  =  -5.10737
	[1  ,   5]  =  -4.51714
	[1  ,   6]  =  -8.55479
	[1  ,   7]  =  -16.2465
	[1  ,   8]  =  -14.5636
	[1  ,   9]  =  -6.82135
	[1  ,  10]  =  -3.11045
	[1  ,  11]  =  -6.56127
	[1  ,  12]  =  -6.46561
	⋮
	[85 , 102]  =  -2.48491
	[94 , 102]  =  -2.45292
	[12 , 103]  =  -10.6282
	[31 , 103]  =  -8.12024
	[40 , 103]  =  -7.75982
	[49 , 103]  =  -7.14859
	[58 , 103]  =  -7.10347
	[67 , 103]  =  -7.31459
	[76 , 103]  =  -7.63411
	[85 , 103]  =  -7.50893
	[94 , 103]  =  -8.47637

469729891 seconds
length(unordered_markers) = 13
length(m.linconstr) = 10205


In [ ]:
tic()
solve(m)
time_to_solve = toc()
println("Objective value: ", getObjectiveValue(m))
println("Prob: ", e^getObjectiveValue(m))
x_val = getValue(x)
#println("x = ", x_val)
x_iis,x_jjs, _  = findnz(x_val.>1-1e-6)
zip(x_iis,x_jjs) |> collect |> println

In [ ]:
time_to_solve

In [ ]:
nodes_chain = Dict([ii=>jj for (ii,jj) in zip(x_iis,x_jjs)])
node_index=1
visitted = Set{Int}()
while(node_index!=END_NODE_INDEX)
    push!(visitted, node_index)
    println(unordered_markers[nodes[node_index][1]])
    #println(node_index, " ", unordered_markers[nodes[node_index][1]]," ", unordered_markers[nodes[node_index][2]])
    node_index = nodes_chain[node_index]
end

In [ ]:
x_val = getValue(x)
x_iis,x_jjs, _  = findnz(x_val)
nodes_chain = Dict([ii=>jj for (ii,jj) in zip(x_iis,x_jjs)])

In [ ]:
x

In [ ]:
collect(1:100)[[1]]

In [ ]:
sts = get_subtours(x)

In [ ]:
1+1

In [ ]:
for tran in union(banned_trans_prob, banned_trans_state)
    for rule in values(rules)
        delete!(rule, tran)
    end
end
rules

In [ ]:
rules["fine*"] ∩ keys(nodes_chain)

In [ ]:
function get_prod(x, iis, jjs)
    net = 1.0
    for i in 1:size(x,1) 
        for j in 1:size(x,2)
            println(i,",", j, " =", x[i,j], " ", trans_prob[i,j])
            net*=max((x[i,j]-1)^2, trans_prob[i,j])
        end
    end
    net
end
values = falses(size(x))
values[1,3] = 1
values[3,6] = 1
values[6,7] = 1
println("----------\n")
get_prod(values, iis,jjs)


In [ ]:
@pyimport nltk.corpus as nltk_corpus
corpus_reader=nltk_corpus.brown
corpus = Vector{ASCIIString}[[lowercase(word) for word in sent] for sent in (corpus_reader[:sents]()|> collect)]
const log_lm = train_language_model(corpus, loglikelyhood=true)